In [35]:
# ===================================================================
#                             2ESR
#          558385 - Alexia Ramalho Izidio Dos Santos
#          554746 - Beatriz Vieira de Novais
#          559008 - Hellen Aparecida Moura Silva
#          557397 - Lorenzo Adinolfi Acquesta
#          558859 - Wendell Dos Santos Silva
# ===================================================================

Dinâmica de Consumo de Insumos — Simulação e Estruturas/Algoritmos
Neste notebook:

- Gera dados simulados de insumos (cada insumo tem data de validade).
- Simula consumo diário (registros cronológicos).
- Implementa Fila (ordem cronológica) e Pilha (consultas em ordem inversa).
- Implementa Busca Sequencial e Busca Binária.
- Implementa Merge Sort e Quick Sort (ordenar por quantidade consumida ou validade).
- Mostra tabelas após cada etapa.


In [36]:
import random
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from collections import deque
from IPython.display import display

random.seed(123)  # reprodutibilidade
np.random.seed(123)


In [37]:
# usamos um catálogo de insumos onde cada insumo tem validade
n_items = 12
base_date = datetime.today().date()

# cria os nomes fixos para melhor visualização
nomes = [
    "Reagente A", "Reagente B", "Reagente C", "Reagente D",
    "Cartucho X", "Cartucho Y", "Ponta 200uL", "Ponta 1000uL",
    "Tubo 15mL", "Tubo 50mL", "Luvas P", "Luvas M"
]

insumos = []
for i in range(n_items):
    item = {
        "item_id": f"ITM{i+1:03d}",
        "nome": nomes[i],
        # validade aleatória entre 30 e 540 dias a partir de hoje
        "validade": base_date + timedelta(days=random.randint(30, 540)),
        # estoque inicial aleatório
        "estoque_inicial": random.randint(10, 200)
    }
    insumos.append(item)

df_insumos = pd.DataFrame(insumos)
df_insumos["validade"] = pd.to_datetime(df_insumos["validade"]).dt.date
display(df_insumos)


,item_id,nome,validade,estoque_inicial
0,ITM001,Reagente A,2025-11-09,78
1,ITM002,Reagente B,2025-11-27,114
2,ITM003,Reagente C,2026-02-27,37
3,ITM004,Reagente D,2026-12-17,19
4,ITM005,Cartucho X,2026-04-26,147
5,ITM006,Cartucho Y,2026-07-28,95
6,ITM007,Ponta 200uL,2026-04-06,23
7,ITM008,Ponta 1000uL,2026-01-03,44
8,ITM009,Tubo 15mL,2026-04-04,153
9,ITM010,Tubo 50mL,2026-04-02,189


In [39]:
# simula o consumo diário por registros cronológicos
dias = 60  # simular 60 dias de consumo
start_date = base_date - timedelta(days=dias-1)  # começa dias-1 atrás
consumos = []

for d in range(dias):
    data = start_date + timedelta(days=d)
    # em cada dia, consome entre 1 e 5 itens diferentes
    n_consume = random.randint(1, 5)
    itens_hoje = random.sample(insumos, n_consume)
    for it in itens_hoje:
        # quantidade consumida por dia por insumo (0 a 5)
        qtd = random.randint(1, 6)
        consumos.append({
            "data": data,
            "item_id": it["item_id"],
            "nome": it["nome"],
            "qtd": qtd,
            "validade": it["validade"]
        })

df_consumo = pd.DataFrame(consumos)
df_consumo["data"] = pd.to_datetime(df_consumo["data"]).dt.date
# garantir ordenação cronológica
df_consumo = df_consumo.sort_values(by="data").reset_index(drop=True)
display(df_consumo.head(30))  # mostra primeiros 30 registros (tabela)


,data,item_id,nome,qtd,validade
0,2025-07-17,ITM012,Luvas M,4,2025-10-14
1,2025-07-17,ITM002,Reagente B,2,2025-11-27
2,2025-07-17,ITM004,Reagente D,4,2026-12-17
3,2025-07-18,ITM008,Ponta 1000uL,1,2026-01-03
4,2025-07-18,ITM003,Reagente C,2,2026-02-27
5,2025-07-19,ITM006,Cartucho Y,1,2026-07-28
6,2025-07-20,ITM003,Reagente C,2,2026-02-27
7,2025-07-21,ITM012,Luvas M,1,2025-10-14
8,2025-07-21,ITM001,Reagente A,4,2025-11-09
9,2025-07-21,ITM007,Ponta 200uL,1,2026-04-06


**Fila e Pilha**

• Implementar uma fila para registrar o consumo diário (ordem cronológica).

• Implementar uma pilha para simular consultas em ordem inversa (últimos consumos primeiro).

In [41]:
# implementação de Fila e Pilha
#fila
class Fila:
    def __init__(self):
        self._data = deque()
    def enqueue(self, item):
        self._data.append(item)
    def dequeue(self):
        if not self.is_empty():
            return self._data.popleft()
        return None
    def is_empty(self):
        return len(self._data) == 0
    def size(self):
        return len(self._data)
    def to_list(self):
        return list(self._data)

#pilha
class Pilha:
    def __init__(self):
        self._data = []
    def push(self, item):
        self._data.append(item)
    def pop(self):
        if not self.is_empty():
            return self._data.pop()
        return None
    def is_empty(self):
        return len(self._data) == 0
    def size(self):
        return len(self._data)
    def to_list(self):
        # mostra de topo para base (último elemento primeiro)
        return list(reversed(self._data))


In [42]:
# usa fila para registrar consumo diário por ordem cronológica
fila_consumo = Fila()
for idx, row in df_consumo.iterrows():
    # cada registro é um dicionário simples
    fila_consumo.enqueue(row.to_dict())

# converte para DataFrame para exibição
df_fila = pd.DataFrame(fila_consumo.to_list())
display(df_fila.head(20))
print("Tamanho da fila:", fila_consumo.size())


,data,item_id,nome,qtd,validade
0,2025-07-17,ITM012,Luvas M,4,2025-10-14
1,2025-07-17,ITM002,Reagente B,2,2025-11-27
2,2025-07-17,ITM004,Reagente D,4,2026-12-17
3,2025-07-18,ITM008,Ponta 1000uL,1,2026-01-03
4,2025-07-18,ITM003,Reagente C,2,2026-02-27
5,2025-07-19,ITM006,Cartucho Y,1,2026-07-28
6,2025-07-20,ITM003,Reagente C,2,2026-02-27
7,2025-07-21,ITM012,Luvas M,1,2025-10-14
8,2025-07-21,ITM001,Reagente A,4,2025-11-09
9,2025-07-21,ITM007,Ponta 200uL,1,2026-04-06


Tamanho da fila: 175


In [43]:
# usa pilha para consultas em ordem inversa onde os últimos consumos vem primeiro
pilha_consumo = Pilha()
for idx, row in df_consumo.iterrows():
    pilha_consumo.push(row.to_dict())

# mostra os 10 "topo" (últimos consumos)
top_n = 10
top_items = []
for _ in range(min(top_n, pilha_consumo.size())):
    item = pilha_consumo.pop()
    top_items.append(item)

df_top = pd.DataFrame(top_items)
display(df_top)  # esses são os últimos consumos (em ordem do mais recente para o menos recente)
print("Tamanho da pilha após pops:", pilha_consumo.size())


,data,item_id,nome,qtd,validade
0,2025-09-14,ITM002,Reagente B,5,2025-11-27
1,2025-09-13,ITM006,Cartucho Y,2,2026-07-28
2,2025-09-12,ITM006,Cartucho Y,3,2026-07-28
3,2025-09-12,ITM005,Cartucho X,5,2026-04-26
4,2025-09-11,ITM007,Ponta 200uL,6,2026-04-06
5,2025-09-11,ITM011,Luvas P,1,2026-02-16
6,2025-09-11,ITM004,Reagente D,6,2026-12-17
7,2025-09-11,ITM008,Ponta 1000uL,3,2026-01-03
8,2025-09-10,ITM006,Cartucho Y,2,2026-07-28
9,2025-09-10,ITM008,Ponta 1000uL,2,2026-01-03


Tamanho da pilha após pops: 165


**Estruturas de Busca**

• Implementar busca sequencial e binária para localizar um insumo específico no registro de consumo.

In [45]:
# busca sequencial em registros de consumo
def busca_sequencial(df_records, chave, valor):
    # retorna lista de índices/linhas que correspondem
    results = []
    for idx, row in df_records.iterrows():
        if row[chave] == valor:
            results.append(row.to_dict())
    return results

# por exemplo: buscar por nome "Reagente A"
nome_busca = "Reagente A"
res_seq = busca_sequencial(df_consumo, "nome", nome_busca)
df_res_seq = pd.DataFrame(res_seq)
print(f"Registros encontrados (busca sequencial) para '{nome_busca}': {len(res_seq)}")
display(df_res_seq.head(20))


Registros encontrados (busca sequencial) para 'Reagente A': 12


,data,item_id,nome,qtd,validade
0,2025-07-21,ITM001,Reagente A,4,2025-11-09
1,2025-07-29,ITM001,Reagente A,1,2025-11-09
2,2025-08-02,ITM001,Reagente A,6,2025-11-09
3,2025-08-07,ITM001,Reagente A,5,2025-11-09
4,2025-08-10,ITM001,Reagente A,2,2025-11-09
5,2025-08-14,ITM001,Reagente A,2,2025-11-09
6,2025-08-17,ITM001,Reagente A,4,2025-11-09
7,2025-08-19,ITM001,Reagente A,5,2025-11-09
8,2025-08-24,ITM001,Reagente A,5,2025-11-09
9,2025-08-27,ITM001,Reagente A,5,2025-11-09


In [46]:
# busca binária: requer dados ordenados por uma chave.
# aqui vamos preparar uma lista de insumos únicos ordenada por 'nome' e implementar busca binária por nome.

# Lista de insumos únicos (do catálogo)
lista_insumos = df_insumos.sort_values(by="nome").to_dict(orient="records")
df_lista_insumos = pd.DataFrame(lista_insumos)
display(df_lista_insumos)  # já ordenada por 'nome'

# implementa a busca binária por nome na lista_insumos
def busca_binaria(lista, chave, valor):
    lo, hi = 0, len(lista) - 1
    while lo <= hi:
        mid = (lo + hi) // 2
        mid_val = lista[mid][chave]
        if mid_val == valor:
            return lista[mid]  # retorna o dicionário do insumo
        elif mid_val < valor:
            lo = mid + 1
        else:
            hi = mid - 1
    return None

# Exemplo de uso
nome_procurado = "Ponta 200uL"
res_bin = busca_binaria(lista_insumos, "nome", nome_procurado)
print("Resultado busca binária por nome:", res_bin)


,item_id,nome,validade,estoque_inicial
0,ITM005,Cartucho X,2026-04-26,147
1,ITM006,Cartucho Y,2026-07-28,95
2,ITM012,Luvas M,2025-10-14,121
3,ITM011,Luvas P,2026-02-16,51
4,ITM008,Ponta 1000uL,2026-01-03,44
5,ITM007,Ponta 200uL,2026-04-06,23
6,ITM001,Reagente A,2025-11-09,78
7,ITM002,Reagente B,2025-11-27,114
8,ITM003,Reagente C,2026-02-27,37
9,ITM004,Reagente D,2026-12-17,19


Resultado busca binária por nome: {'item_id': 'ITM007', 'nome': 'Ponta 200uL', 'validade': datetime.date(2026, 4, 6), 'estoque_inicial': 23}


In [29]:
# agregar total consumido por insumo (para ordenação por quantidade)
df_total = df_consumo.groupby(["item_id", "nome", "validade"], as_index=False)["qtd"].sum()
df_total = df_total.rename(columns={"qtd": "total_consumido"})
display(df_total.sort_values(by="total_consumido", ascending=False).reset_index(drop=True))


,item_id,nome,validade,total_consumido
0,ITM007,Ponta 200uL,2026-04-06,89
1,ITM001,Reagente A,2025-11-09,72
2,ITM011,Luvas P,2026-02-16,62
3,ITM009,Tubo 15mL,2026-04-04,60
4,ITM003,Reagente C,2026-02-27,58
5,ITM006,Cartucho Y,2026-07-28,58
6,ITM004,Reagente D,2026-12-17,51
7,ITM005,Cartucho X,2026-04-26,46
8,ITM010,Tubo 50mL,2026-04-02,45
9,ITM008,Ponta 1000uL,2026-01-03,45


**Ordenação**

• Implementar algoritmos de ordenação (Merge Sort e Quick Sort) para organizar os insumos por quantidade
consumida ou validade.

In [30]:
# implementa Merge Sort e Quick Sort em listas de dicionários
# essas funções retornam nova lista ordenada com base em uma chave (e opcional reverse)

def merge_sort(arr, chave, reverse=False):
    if len(arr) <= 1:
        return arr[:]
    mid = len(arr) // 2
    left = merge_sort(arr[:mid], chave, reverse)
    right = merge_sort(arr[mid:], chave, reverse)
    merged = []
    i = j = 0
    while i < len(left) and j < len(right):
        if reverse:
            cond = left[i][chave] > right[j][chave]
        else:
            cond = left[i][chave] <= right[j][chave]
        if cond:
            merged.append(left[i])
            i += 1
        else:
            merged.append(right[j])
            j += 1
    merged.extend(left[i:])
    merged.extend(right[j:])
    return merged

def quick_sort(arr, chave, reverse=False):
    if len(arr) <= 1:
        return arr[:]
    pivot = arr[len(arr) // 2][chave]
    less = [x for x in arr if (x[chave] < pivot) ^ reverse]
    equal = [x for x in arr if x[chave] == pivot]
    greater = [x for x in arr if (x[chave] > pivot) ^ reverse]
    return quick_sort(less, chave, reverse) + equal + quick_sort(greater, chave, reverse)

# prepara a lista de dicionários a partir do df_total
lista_total = df_total.to_dict(orient="records")

# Merge Sort por total_consumido (descendente) - top consumos primeiro
ms_sorted_qty = merge_sort(lista_total, "total_consumido", reverse=True)
df_ms_qty = pd.DataFrame(ms_sorted_qty)
print("Merge Sort - ordenado por total_consumido (desc):")
display(df_ms_qty.head(15))

# Quick Sort por validade (ascendente) - validade mais próxima primeiro
# convertendo validade para datas comparáveis (se ainda for date, ok)
# garante que cada registro tenha validade como date
for rec in lista_total:
    if isinstance(rec["validade"], pd.Timestamp):
        rec["validade"] = rec["validade"].date()

qs_sorted_val = quick_sort(lista_total, "validade", reverse=False)
df_qs_val = pd.DataFrame(qs_sorted_val)
print("Quick Sort - ordenado por validade (asc):")
display(df_qs_val.head(15))


Merge Sort - ordenado por total_consumido (desc):


,item_id,nome,validade,total_consumido
0,ITM007,Ponta 200uL,2026-04-06,89
1,ITM001,Reagente A,2025-11-09,72
2,ITM011,Luvas P,2026-02-16,62
3,ITM009,Tubo 15mL,2026-04-04,60
4,ITM006,Cartucho Y,2026-07-28,58
5,ITM003,Reagente C,2026-02-27,58
6,ITM004,Reagente D,2026-12-17,51
7,ITM005,Cartucho X,2026-04-26,46
8,ITM010,Tubo 50mL,2026-04-02,45
9,ITM008,Ponta 1000uL,2026-01-03,45


Quick Sort - ordenado por validade (asc):


,item_id,nome,validade,total_consumido
0,ITM012,Luvas M,2025-10-14,39
1,ITM001,Reagente A,2025-11-09,72
2,ITM002,Reagente B,2025-11-27,34
3,ITM008,Ponta 1000uL,2026-01-03,45
4,ITM011,Luvas P,2026-02-16,62
5,ITM003,Reagente C,2026-02-27,58
6,ITM010,Tubo 50mL,2026-04-02,45
7,ITM009,Tubo 15mL,2026-04-04,60
8,ITM007,Ponta 200uL,2026-04-06,89
9,ITM005,Cartucho X,2026-04-26,46


In [47]:
# busca binária em lista ordenada por 'nome' já foi mostrada.
# por exemplo: se quisermos fazer busca binária por 'item_id' precisamos de lista ordenada por 'item_id'.

lista_por_id = sorted(lista_total, key=lambda x: x["item_id"])
res = busca_binaria(lista_por_id, "item_id", "ITM005")
print("Busca binária por item_id 'ITM005':")
print(res)


Busca binária por item_id 'ITM005':
{'item_id': 'ITM005', 'nome': 'Cartucho X', 'validade': datetime.date(2026, 4, 26), 'total_consumido': 46}


**Relatório** [SPRINT3_Dynamic_Programming](https://github.com/hellen-silvaa/SPRINT3_Dynamic_Programming)